# AI Tutor - Textbook Ingestion (Google Colab)

This notebook uses the AI Tutor's ingestion system to process textbooks.

**Features:**
- ✅ Uses TutorSystem ingestion pipeline
- ✅ Upload your own PDFs or use sample textbooks
- ✅ Creates embeddings with sentence-transformers
- ✅ **Automatic domain classification** (math, physics, cs, chemistry, biology, general)
- ✅ **Domain-based collections** in ChromaDB for better organization
- ✅ AI-powered domain detection for accurate classification
- ✅ Works with `chroma_data/chroma.sqlite3` database
- ✅ Save/load database from Google Drive

**Domain Classification:**
Documents are automatically classified into domains based on content. Each domain gets its own ChromaDB collection for efficient organization and retrieval.


In [3]:
print("🔄 Cloning AI Tutor repository...")
!git clone -b mcp_server https://github.com/HenryNVP/ai-tutor.git
%cd ai-tutor

!pip install -r requirements.txt

🔄 Cloning AI Tutor repository...
Cloning into 'ai-tutor'...
remote: Enumerating objects: 752, done.
remote: Counting objects: 100% (752/752), done.
remote: Compressing objects: 100% (386/386), done.
remote: Total 752 (delta 380), reused 672 (delta 311), pack-reused 0 (from 0)
Receiving objects: 100% (752/752), 842.21 KiB | 4.01 MiB/s, done.
Resolving deltas: 100% (380/380), done.
/content/ai-tutor/ai-tutor
Obtaining file:///content/ai-tutor/ai-tutor (from -r requirements.txt (line 1))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.2 MB/s eta 0:00:00
     ━

In [1]:
import os
from pathlib import Path

%cd ai-tutor

PROJECT_ROOT = Path.cwd()
print(f"Project root: {PROJECT_ROOT}")

import sys
SRC_ROOT = PROJECT_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))


/content/ai-tutor
Project root: /content/ai-tutor


In [4]:
# Initialize TutorSystem with chroma_data directory
from ai_tutor.system import TutorSystem
from ai_tutor.config.loader import load_settings
from pathlib import Path
import os
from google.colab import userdata

# Optional: Set API key
os.environ["OPENAI_DEFAULT_MODEL"]="gpt-4o-mini"
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

print("🚀 Loading settings...")
settings = load_settings()

# Override vector_store_dir to use chroma_data instead of data/vector_store
settings.paths.vector_store_dir = Path("chroma_data")
print(f"  Vector store directory: {settings.paths.vector_store_dir}")

# Create chroma_data directory if it doesn't exist
chroma_data_dir = PROJECT_ROOT / "chroma_data"
chroma_data_dir.mkdir(exist_ok=True)
print(f"  Created/verified chroma_data directory: {chroma_data_dir}")

print("\n🚀 Initializing TutorSystem...")
system = TutorSystem(settings=settings)

print(f"\n✓ TutorSystem initialized")
print(f"  Embedding model: {system.embedder.config.model}")
print(f"  Chunk size: {system.settings.chunking.chunk_size}")
print(f"  Vector store: {system.settings.paths.vector_store_dir}")
print(f"  Database location: {chroma_data_dir}/chroma.sqlite3")


🚀 Loading settings...
  Vector store directory: chroma_data
  Created/verified chroma_data directory: /content/ai-tutor/chroma_data

🚀 Initializing TutorSystem...

✓ TutorSystem initialized
  Embedding model: BAAI/bge-base-en
  Chunk size: 500
  Vector store: chroma_data
  Database location: /content/ai-tutor/chroma_data/chroma.sqlite3


In [7]:
from google.colab import drive

drive.mount('/content/drive')
# Data folder in Drive
drive_folder_path = '/content/drive/MyDrive/ai-tutor/raw'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Upload PDFs from Google Drive to domain-specific folders in data/raw
from google.colab import files
import shutil

# Define project directories
raw_data_dir = PROJECT_ROOT / "data" / "raw"

# Define domain subfolders
domains = ["math", "physics", "cs", "general"]
for domain in domains:
    (raw_data_dir / domain).mkdir(parents=True, exist_ok=True)

# Copy PDF files from Google Drive folders to corresponding domain folders
print(f"Copying PDF files from '{drive_folder_path}' into domain folders in '{raw_data_dir}'...")

for domain in domains:
    domain_folder = Path(drive_folder_path) / domain
    if not domain_folder.exists():
        print(f"⚠️ Drive folder for domain '{domain}' does not exist, skipping...")
        continue

    for pdf_file in domain_folder.glob("*.pdf"):
        dest_file = raw_data_dir / domain / pdf_file.name
        shutil.copy(pdf_file, dest_file)
        print(f"✅ {domain}/{pdf_file.name}")

# List all PDFs in all domain folders
print("\n📚 Found PDF files ready for ingestion:")
for domain in domains:
    domain_files = sorted((raw_data_dir / domain).glob("*.pdf"))
    if domain_files:
        print(f"\n--- {domain.upper()} ---")
        for i, pdf in enumerate(domain_files, 1):
            size_mb = pdf.stat().st_size / (1024 * 1024)
            print(f"  {i}. {pdf.name} ({size_mb:.1f} MB)")
    else:
        print(f"\n--- {domain.upper()} --- No files found")

Copying PDF files from '/content/drive/MyDrive/ai-tutor/raw' to '/content/ai-tutor/data/raw'...
✅ collegephysicsvol1.pdf
✅ collegephysicsvol3.pdf
✅ collegephysicsvol2.pdf
✅ Algebra-and-Trigonometry-2e-WEB.pdf
✅ Calculus_Volume_1_-_WEB_68M1Z5W.pdf
✅ Principles-of-Data-Science-WEB.pdf
✅ Introduction_To_Computer_Science_-_WEB.pdf

📚 Found 7 PDF file(s) ready for ingestion:

  1. Algebra-and-Trigonometry-2e-WEB.pdf (109.6 MB)
  2. Calculus_Volume_1_-_WEB_68M1Z5W.pdf (44.5 MB)
  3. Introduction_To_Computer_Science_-_WEB.pdf (50.9 MB)
  4. Principles-of-Data-Science-WEB.pdf (32.4 MB)
  5. collegephysicsvol1.pdf (40.7 MB)
  6. collegephysicsvol2.pdf (35.2 MB)
  7. collegephysicsvol3.pdf (29.5 MB)


In [9]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 108.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chroma-mcp-server 0.2.28 requires transformers~=4.41.0, but you have transformers 4.57.1 which is incompatible.


In [2]:
!pip show transformers sentence-transformers

Name: transformers
Version: 4.57.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: chroma-mcp-server, peft, sentence-transformers
---
Name: sentence-transformers
Version: 5.1.2
Summary: Embeddings, Retrieval, and Reranking
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers, typing_extensio

In [9]:
# Ingest all textbooks
# This will:
# 1. Parse PDFs
# 2. Chunk the text
# 3. Generate embeddings
# 4. Store in vector store

print("🔄 Starting ingestion (this may take a few minutes)...\n")

result = system.ingest_directory(raw_data_dir)

print("\n" + "="*60)
print("✅ INGESTION COMPLETE")
print("="*60)
print(f"📄 Documents processed: {len(result.documents)}")
print(f"📝 Chunks created: {len(result.chunks)}")
print(f"⏭️  Files skipped: {len(result.skipped)}")

if result.skipped:
    print(f"\nSkipped files:")
    for skip in result.skipped:
        print(f"  - {skip}")


🔄 Starting ingestion (this may take a few minutes)...



Ingesting documents: 100%|██████████| 7/7 [00:13<00:00,  1.96s/it]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ INGESTION COMPLETE
📄 Documents processed: 7
📝 Chunks created: 4678
⏭️  Files skipped: 0


In [ ]:
# Display domain classification results
print("📊 Domain Classification Results:\n")
print("="*60)

from collections import Counter

# Group documents by domain
domain_stats = Counter()
domain_chunks = Counter()

for doc in result.documents:
    primary = getattr(doc.metadata, "primary_domain", doc.metadata.domain or "general")
    domain_stats[primary] += 1
    
    # Count chunks per domain
    doc_chunks = [c for c in result.chunks if c.metadata.doc_id == doc.metadata.doc_id]
    domain_chunks[primary] += len(doc_chunks)

# Display classification details
for doc in result.documents:
    primary = getattr(doc.metadata, "primary_domain", doc.metadata.domain or "general")
    secondary = getattr(doc.metadata, "secondary_domains", []) or []
    tags = getattr(doc.metadata, "domain_tags", []) or []
    confidence = getattr(doc.metadata, "domain_confidence", 0.5)
    
    print(f"\n📄 {doc.metadata.source_path.name}")
    print(f"   Primary Domain: {primary}")
    if secondary:
        print(f"   Secondary Domains: {', '.join(secondary)}")
    if tags:
        print(f"   Tags: {', '.join(tags[:3])}")  # Show first 3 tags
    print(f"   Confidence: {confidence:.2f}")

# Display summary statistics
print("\n" + "="*60)
print("📈 Summary by Domain:")
print("="*60)
for domain, doc_count in sorted(domain_stats.items()):
    chunk_count = domain_chunks[domain]
    print(f"  {domain:12s}: {doc_count:2d} documents, {chunk_count:5d} chunks")

# Check ChromaDB collections
print("\n" + "="*60)
print("🗂️  ChromaDB Collections:")
print("="*60)

try:
    from ai_tutor.retrieval.chroma_store import ChromaVectorStore
    import chromadb
    
    # Check if using domain collections
    if hasattr(system.vector_store, 'use_domain_collections') and system.vector_store.use_domain_collections:
        print("  ✓ Using domain-based collections")
        
        # List all collections
        client = system.vector_store.client
        collections = client.list_collections()
        
        for collection_obj in collections:
            collection = client.get_collection(name=collection_obj.name)
            count = collection.count()
            domain = collection_obj.metadata.get("domain", "unknown") if collection_obj.metadata else "unknown"
            print(f"    • {collection_obj.name:25s}: {count:5d} chunks (domain: {domain})")
    else:
        print("  ℹ️  Using single collection (legacy mode)")
        if hasattr(system.vector_store, 'collection'):
            count = system.vector_store.collection.count()
            print(f"    • Collection: {count} chunks")
except Exception as e:
    print(f"  ⚠️  Could not list collections: {e}")

print("\n✅ Domain classification complete!")


In [10]:
# Verify ingestion
print("📊 Verification:\n")

# Check chunks file
chunks_file = PROJECT_ROOT / "data" / "processed" / "chunks.jsonl"
if chunks_file.exists():
    with open(chunks_file, 'r') as f:
        chunk_count = sum(1 for _ in f)
    print(f"✓ Chunks stored: {chunk_count}")
else:
    print("✗ No chunks file found")

# Check Chroma database
chroma_db_path = PROJECT_ROOT / "chroma_data" / "chroma.sqlite3"
if chroma_db_path.exists():
    import sqlite3
    conn = sqlite3.connect(chroma_db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    print(f"✓ Chroma database exists: {chroma_db_path}")
    print(f"  Tables: {len(tables)}")
else:
    print("✗ No Chroma database found")

# Check for collection directories
chroma_data_dir = PROJECT_ROOT / "chroma_data"
collection_dirs = [d for d in chroma_data_dir.iterdir() if d.is_dir() and d.name != 'chroma_example']
if collection_dirs:
    print(f"✓ Collection directories: {len(collection_dirs)}")
else:
    print("ℹ️  No collection directories yet (will be created on first use)")

print("\n✅ Ingestion successful!")


📊 Verification:

✓ Chunks stored: 4678
✓ Chroma database exists: /content/ai-tutor/chroma_data/chroma.sqlite3
  Tables: 20
✓ Collection directories: 1

✅ Ingestion successful!


In [11]:
folders_to_download = [
    PROJECT_ROOT / "data" / "processed",
    PROJECT_ROOT / "data" / "vector_store"
]

for folder_to_download in folders_to_download:
    if folder_to_download.exists():
        zip_filename = f"{folder_to_download.name}.zip"
        # Create a zip archive of the folder
        print(f"Compressing '{folder_to_download}' into '{zip_filename}'...")
        shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', folder_to_download)

        # Download the zip file
        print(f"Downloading '{zip_filename}'...")
        files.download(zip_filename)
        print(f"✅ Download initiated for {zip_filename}")
    else:
        print(f"❗️ Folder not found: {folder_to_download}. Skipping download for this folder.")

print("\nFinished processing download requests.")

Compressing '/content/ai-tutor/data/processed' into 'processed.zip'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download initiated for processed.zip
❗️ Folder not found: /content/ai-tutor/data/vector_store. Skipping download for this folder.

Finished processing download requests.


In [12]:
# Save database to Google Drive
chroma_db_path = PROJECT_ROOT / 'chroma_data' / 'chroma.sqlite3'
drive_db_dir = Path('/content/drive/MyDrive/ai-tutor/chroma_data')
drive_db_dir.mkdir(parents=True, exist_ok=True)
drive_db_path = drive_db_dir / 'chroma.sqlite3'

if chroma_db_path.exists():
    print(f"💾 Saving database to Drive...")
    shutil.copy(chroma_db_path, drive_db_path)
    size_mb = chroma_db_path.stat().st_size / (1024 * 1024)
    print(f"✅ Database saved to Drive")
    print(f"   Size: {size_mb:.2f} MB")
    print(f"   Location: {drive_db_path}")

    # Also save collection directories if they exist
    chroma_data_dir = PROJECT_ROOT / 'chroma_data'
    for item in chroma_data_dir.iterdir():
        if item.is_dir() and item.name != 'chroma_example':
            drive_collection_dir = drive_db_dir / item.name
            if drive_collection_dir.exists():
                shutil.rmtree(drive_collection_dir)
            shutil.copytree(item, drive_collection_dir)
            print(f"   ✅ Saved collection directory: {item.name}")
else:
    print("⚠️  No database found to save")


💾 Saving database to Drive...
✅ Database saved to Drive
   Size: 126.38 MB
   Location: /content/drive/MyDrive/ai-tutor/chroma_data/chroma.sqlite3
   ✅ Saved collection directory: b7055f6e-1c9c-4951-88f0-3ae07efa5d19


In [13]:
import shutil
from pathlib import Path

processed_folder_path = PROJECT_ROOT / "data" / "processed"
drive_processed_dir = Path('/content/drive/MyDrive/ai-tutor/processed')

if processed_folder_path.exists():
    print(f"💾 Saving processed data to Drive...")
    if drive_processed_dir.exists():
        shutil.rmtree(drive_processed_dir)  # Remove existing folder to avoid errors
    shutil.copytree(processed_folder_path, drive_processed_dir)
    print(f"✅ Processed data saved to Drive at: {drive_processed_dir}")
else:
    print("⚠️  Processed data folder not found to save.")

💾 Saving processed data to Drive...
✅ Processed data saved to Drive at: /content/drive/MyDrive/ai-tutor/processed
